In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,963 kB]


In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demographicsFilter").getOrCreate()

In [ ]:
from pyspark.sql.functions import mean, stddev
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
import numpy as np
import pandas as pd

column_counts=[1, 10, 100, 1000]
# element_counts=[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]
row_counts=[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]
results=[]
for each_row_count in row_counts: 
  for each_column_count in column_counts: 
    each_element_count=each_row_count*each_column_count
    if each_element_count<=10000000: 
      print(f'{each_column_count} by {each_row_count}')
      result={}
      result['element_count']=each_element_count
      result['column_count']=each_column_count
      ary=np.random.randint(100, size=each_element_count)
      array_time=%timeit -o -n 3 np.random.randint(100, size=each_element_count)
      result['array_time']=array_time.best
      ary=ary.reshape(-1, each_column_count)
      df=pd.DataFrame(ary)
      df_time=%timeit -o -n 3 pd.DataFrame(ary)
      result['pandas_df_time']=df_time.best
      df_compute=%timeit -o -n 3 df.mean()
      result['pandas_df_compute']=df_compute.best
      spark_df=spark.createDataFrame(df)
      spark_df_time=%timeit -o -n 3 spark.createDataFrame(df)
      result['spark_df_time']=spark_df_time.best
      spark_df_compute=%timeit -o -n 3 spark_df.select(*[mean(x) for x in spark_df.columns]).collect()
      # spark_df_compute=%timeit -o -n 3 spark_df.corr()
      
      # # convert to vector column first
      # if not each_row_count==1: 
      #   vector_col = "corr_features"
      #   assembler = VectorAssembler(inputCols=spark_df.columns, outputCol=vector_col)
      #   assembler_time=%timeit -o -n 3 VectorAssembler(inputCols=spark_df.columns, outputCol=vector_col)
      #   df_vector = assembler.transform(spark_df).select(vector_col)
      #   transform_time=%timeit -o -n 3 assembler.transform(spark_df).select(vector_col)
      #   # get correlation matrix
      #   corr_time=%timeit -o -n 3 Correlation.corr(df_vector, vector_col).collect()
      #   spark_df_compute={'best': assembler_time.best+transform_time.best+corr_time.best}

      #   result['spark_df_compute']=spark_df_compute['best']
      
      result['spark_df_compute']=spark_df_compute.best
      results.append(result)
results

1 by 1
3 loops, best of 5: 9.86 µs per loop
3 loops, best of 5: 52.3 µs per loop
3 loops, best of 5: 111 µs per loop
3 loops, best of 5: 14 ms per loop
3 loops, best of 5: 130 ms per loop
10 by 1
3 loops, best of 5: 20 µs per loop
3 loops, best of 5: 94.5 µs per loop
The slowest run took 7.27 times longer than the fastest. This could mean that an intermediate result is being cached.
3 loops, best of 5: 255 µs per loop
3 loops, best of 5: 19.7 ms per loop
3 loops, best of 5: 152 ms per loop
100 by 1
The slowest run took 18.11 times longer than the fastest. This could mean that an intermediate result is being cached.
3 loops, best of 5: 18.4 µs per loop
The slowest run took 21.80 times longer than the fastest. This could mean that an intermediate result is being cached.
3 loops, best of 5: 102 µs per loop
3 loops, best of 5: 644 µs per loop
3 loops, best of 5: 33.4 ms per loop
3 loops, best of 5: 851 ms per loop
1000 by 1
3 loops, best of 5: 19.8 µs per loop
3 loops, best of 5: 52.8 µs p

[{'array_time': 9.857666858200295e-06,
  'column_count': 1,
  'element_count': 1,
  'pandas_df_compute': 0.00011096866667988554,
  'pandas_df_time': 5.230466619347377e-05,
  'spark_df_compute': 0.13002163866682773,
  'spark_df_time': 0.013987588999953005},
 {'array_time': 2.000766714142325e-05,
  'column_count': 10,
  'element_count': 10,
  'pandas_df_compute': 0.0002549443330887395,
  'pandas_df_time': 9.449766609274472e-05,
  'spark_df_compute': 0.15225434099981308,
  'spark_df_time': 0.01968203266672693},
 {'array_time': 1.8446333342581056e-05,
  'column_count': 100,
  'element_count': 100,
  'pandas_df_compute': 0.0006440146668561889,
  'pandas_df_time': 0.00010224833325385892,
  'spark_df_compute': 0.8508290616664453,
  'spark_df_time': 0.03339920266686628},
 {'array_time': 1.976099944537661e-05,
  'column_count': 1000,
  'element_count': 1000,
  'pandas_df_compute': 0.0021500713331382335,
  'pandas_df_time': 5.282733339602904e-05,
  'spark_df_compute': 7.937093276000572,
  'spark

In [2]:
results=[{'array_time': 9.857666858200295e-06,
  'column_count': 1,
  'element_count': 1,
  'pandas_df_compute': 0.00011096866667988554,
  'pandas_df_time': 5.230466619347377e-05,
  'spark_df_compute': 0.13002163866682773,
  'spark_df_time': 0.013987588999953005},
 {'array_time': 2.000766714142325e-05,
  'column_count': 10,
  'element_count': 10,
  'pandas_df_compute': 0.0002549443330887395,
  'pandas_df_time': 9.449766609274472e-05,
  'spark_df_compute': 0.15225434099981308,
  'spark_df_time': 0.01968203266672693},
 {'array_time': 1.8446333342581056e-05,
  'column_count': 100,
  'element_count': 100,
  'pandas_df_compute': 0.0006440146668561889,
  'pandas_df_time': 0.00010224833325385892,
  'spark_df_compute': 0.8508290616664453,
  'spark_df_time': 0.03339920266686628},
 {'array_time': 1.976099944537661e-05,
  'column_count': 1000,
  'element_count': 1000,
  'pandas_df_compute': 0.0021500713331382335,
  'pandas_df_time': 5.282733339602904e-05,
  'spark_df_compute': 7.937093276000572,
  'spark_df_time': 0.1547665586670822},
 {'array_time': 9.870333087746985e-06,
  'column_count': 1,
  'element_count': 10,
  'pandas_df_compute': 0.00010662999981529235,
  'pandas_df_time': 5.172399990745665e-05,
  'spark_df_compute': 0.0981610923330057,
  'spark_df_time': 0.012703439999919889},
 {'array_time': 1.8231000164329696e-05,
  'column_count': 10,
  'element_count': 100,
  'pandas_df_compute': 0.00023647533392553063,
  'pandas_df_time': 9.123799948914287e-05,
  'spark_df_compute': 0.1313119373335212,
  'spark_df_time': 0.015130707333203949},
 {'array_time': 2.8697000137375046e-05,
  'column_count': 100,
  'element_count': 1000,
  'pandas_df_compute': 0.0003214733333152253,
  'pandas_df_time': 5.310566666594241e-05,
  'spark_df_compute': 0.8066491676666677,
  'spark_df_time': 0.03224955433324794},
 {'array_time': 0.00010917633335338905,
  'column_count': 1000,
  'element_count': 10000,
  'pandas_df_compute': 0.002172946666178177,
  'pandas_df_time': 5.2594332980030835e-05,
  'spark_df_compute': 7.344100207333516,
  'spark_df_time': 0.20098946500002057},
 {'array_time': 1.0789333828142844e-05,
  'column_count': 1,
  'element_count': 100,
  'pandas_df_compute': 0.00010608633298640295,
  'pandas_df_time': 5.209966669402396e-05,
  'spark_df_compute': 0.09994382700036415,
  'spark_df_time': 0.01465270966703732},
 {'array_time': 2.9781667156688247e-05,
  'column_count': 10,
  'element_count': 1000,
  'pandas_df_compute': 0.00020007166676805355,
  'pandas_df_time': 5.2755000069737434e-05,
  'spark_df_compute': 0.1270205780001561,
  'spark_df_time': 0.021545966666356737},
 {'array_time': 0.00010918699990725145,
  'column_count': 100,
  'element_count': 10000,
  'pandas_df_compute': 0.0003799430002497199,
  'pandas_df_time': 5.385600040123487e-05,
  'spark_df_compute': 0.7793713153332646,
  'spark_df_time': 0.0775438013333769},
 {'array_time': 0.0010908976667754662,
  'column_count': 1000,
  'element_count': 100000,
  'pandas_df_compute': 0.0023865596667747013,
  'pandas_df_time': 5.359199955516184e-05,
  'spark_df_compute': 7.496396976666195,
  'spark_df_time': 0.6907961736669677},
 {'array_time': 1.9217999579268508e-05,
  'column_count': 1,
  'element_count': 1000,
  'pandas_df_compute': 0.00016150599973722515,
  'pandas_df_time': 5.329566677877059e-05,
  'spark_df_compute': 0.09506315899974045,
  'spark_df_time': 0.028627869333528604},
 {'array_time': 0.00016244133318347545,
  'column_count': 10,
  'element_count': 10000,
  'pandas_df_compute': 0.000247029999930722,
  'pandas_df_time': 8.790099976370887e-05,
  'spark_df_compute': 0.12439447333357141,
  'spark_df_time': 0.07591244466675562},
 {'array_time': 0.0010666983331854378,
  'column_count': 100,
  'element_count': 100000,
  'pandas_df_compute': 0.0005406539997541889,
  'pandas_df_time': 5.338133329739018e-05,
  'spark_df_compute': 0.8230792003329649,
  'spark_df_time': 0.5468502510005541},
 {'array_time': 0.012221430667220071,
  'column_count': 1000,
  'element_count': 1000000,
  'pandas_df_compute': 0.007489079000151833,
  'pandas_df_time': 7.603066660522018e-05,
  'spark_df_compute': 8.195854059666695,
  'spark_df_time': 5.568842550333405},
 {'array_time': 0.00010786799975903705,
  'column_count': 1,
  'element_count': 10000,
  'pandas_df_compute': 0.00012249433348188177,
  'pandas_df_time': 5.3884999942965806e-05,
  'spark_df_compute': 0.0997424619999947,
  'spark_df_time': 0.1688797943334066},
 {'array_time': 0.00108280733365973,
  'column_count': 10,
  'element_count': 100000,
  'pandas_df_compute': 0.00034186533351506415,
  'pandas_df_time': 5.256766720170466e-05,
  'spark_df_compute': 0.11906413333357098,
  'spark_df_time': 0.6182825673331536},
 {'array_time': 0.013865724666781412,
  'column_count': 100,
  'element_count': 1000000,
  'pandas_df_compute': 0.0051111076669864515,
  'pandas_df_time': 8.269999974193827e-05,
  'spark_df_compute': 0.9368196483331607,
  'spark_df_time': 5.116832941999746},
 {'array_time': 0.10242950066640333,
  'column_count': 1000,
  'element_count': 10000000,
  'pandas_df_compute': 0.0411799006669753,
  'pandas_df_time': 5.303133366396651e-05,
  'spark_df_compute': 10.638342913999926,
  'spark_df_time': 54.82509145133372},
 {'array_time': 0.0010654486668499885,
  'column_count': 1,
  'element_count': 100000,
  'pandas_df_compute': 0.0003064969999589569,
  'pandas_df_time': 5.2751332987099886e-05,
  'spark_df_compute': 0.12568549966696688,
  'spark_df_time': 1.5560246843330485},
 {'array_time': 0.010670199333617347,
  'column_count': 10,
  'element_count': 1000000,
  'pandas_df_compute': 0.0032762183333640373,
  'pandas_df_time': 5.2707333452417515e-05,
  'spark_df_compute': 0.281180437000027,
  'spark_df_time': 6.098961575666787},
 {'array_time': 0.0995029456668514,
  'column_count': 100,
  'element_count': 10000000,
  'pandas_df_compute': 0.05634355466645502,
  'pandas_df_time': 5.210233333248956e-05,
  'spark_df_compute': 2.4336869873332034,
  'spark_df_time': 51.54112907600029},
 {'array_time': 0.01064574766678561,
  'column_count': 1,
  'element_count': 1000000,
  'pandas_df_compute': 0.0018490976666119725,
  'pandas_df_time': 5.209233313507866e-05,
  'spark_df_compute': 0.8984983489996617,
  'spark_df_time': 15.552167184000306},
 {'array_time': 0.10220646866703949,
  'column_count': 10,
  'element_count': 10000000,
  'pandas_df_compute': 0.06698298433305656,
  'pandas_df_time': 5.225033297998986e-05,
  'spark_df_compute': 1.9034590610002244,
  'spark_df_time': 61.01635254833369},
 {'array_time': 0.10667965066689551,
  'column_count': 1,
  'element_count': 10000000,
  'pandas_df_compute': 0.01718067533329304,
  'pandas_df_time': 5.2310000076734774e-05,
  'spark_df_compute': 7.524165298667261,
  'spark_df_time': 155.98534123833392}]

In [3]:
import pandas as pd
results_df=pd.DataFrame(results)
results_df.to_csv('results_mean.csv')

In [ ]:
ary=np.random.randint(100, size=10000)
ary=ary.reshape(-1, 1)
df=pd.DataFrame(ary)
spark_df=spark.createDataFrame(df)

# convert to vector column first
vector_col = "corr_features"
%timeit assembler = VectorAssembler(inputCols=spark_df.columns, outputCol=vector_col)
%timeit df_vector = assembler.transform(spark_df).select(vector_col)
# get correlation matrix
%timeit print(Correlation.corr(df_vector, vector_col).collect())
# %timeit Correlation.corr(df_vector, vector_col).collect()
# %timeit Correlation.corr(df_vector, vector_col)


100 loops, best of 5: 2.25 ms per loop
100 loops, best of 5: 12.2 ms per loop
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
[Row(pearson(corr_features)=DenseMatrix(1, 1, [1.0], False))]
1 loop, best of 5: 478 ms per loop


In [ ]:
import numpy as np
import pandas as pd
# # from pyspark.ml.linalg import Vectors
# import pandas as pd
ary=np.random.randint(100, size=10000)
# ary=ary.reshape(-1, 2)
# # vc=map(lambda x: (int(x[0]), Vectors.dense(x[1:])), ary)
# # # vc=map(lambda x: int(x), ary)
# # spark.createDataFrame(vc, schema=['number']).show()
df=pd.DataFrame(ary)#, columns=['number_1', 'number_2'])
spark_df=spark.createDataFrame(df)

In [ ]:
df.count()

0    10000
dtype: int64

In [ ]:
spark_df.describe().show()

+-------+-----------------+-----------------+-----------------+------------------+------------------+---------------+------------------+-----------------+-----------------+------------------+
|summary|                0|                1|                2|                 3|                 4|              5|                 6|                7|                8|                 9|
+-------+-----------------+-----------------+-----------------+------------------+------------------+---------------+------------------+-----------------+-----------------+------------------+
|  count|               10|               10|               10|                10|                10|             10|                10|               10|               10|                10|
|   mean|             72.4|             56.2|             50.3|              41.0|              54.2|           47.3|              59.1|             58.7|             63.9|              42.5|
| stddev|24.50940862426328|34.8514307055

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=spark_df.columns, outputCol=vector_col)
df_vector = assembler.transform(spark_df).select(vector_col)
df_vector.show()
# get correlation matrix
# matrix = Correlation.corr(df_vector, vector_col)

+--------------------+
|       corr_features|
+--------------------+
|[96.0,10.0,47.0,4...|
|[77.0,86.0,42.0,6...|
|[67.0,33.0,45.0,1...|
|[96.0,80.0,87.0,3...|
|[84.0,86.0,86.0,6...|
|[83.0,92.0,39.0,9...|
|[18.0,95.0,12.0,1...|
|[90.0,26.0,30.0,5...|
|[46.0,11.0,88.0,1...|
|[67.0,43.0,27.0,6...|
+--------------------+



In [ ]:
# spark_df=spark.createDataFrame(df)
# # spark_df.show()

In [ ]:
# df_results.best

0.4393859119991248

In [ ]:
# # from pyspark.sql.functions import mean
# columns=spark_df.columns
# means=map(lambda x: mean(x), columns)
# # # print(*means)
# # [mean(x) for x in spark_df.columns]
# spark_df.select(*means).show()

+------+
|avg(0)|
+------+
|49.269|
+------+



In [ ]:
# # import org.apache.spark.sql.functions._

# %%timeit -n 1
from pyspark.sql.functions import mean, median, max
# # spark_df.describe().show()
# spark_df.select([mean('0'), mean('0')]).show()
spark_df.select(*[median(x) for x in spark_df.columns]).show()

ImportError: ignored